In [1]:
import numpy as np

In [152]:

def make_contreras_snail(label="snail", 
                         b = .1, d = 4, z = 0, a = 1, phi = 0, psi = 0, 
                         c_depth=0.1, c_n = 70, n_depth = 0, n = 0, 
                         time = 20, n_points_time = 10000, 
                         n_points_aperture=1000):

    _lambda: np.array = np.zeros((3, n_points_time, n_points_aperture))
    gamma: np.array = np.zeros((3, n_points_time, n_points_aperture))
    C: np.array = np.zeros((3, n_points_time, n_points_aperture))

    # vector of points in time reshaped for broadcasting
    t: np.array = np.linspace(0, time, n_points_time).reshape((n_points_time, 1))
    theta: np.array = np.linspace(0, 2*np.pi, n_points_aperture).reshape((1, n_points_aperture))

    # precalculating some repeated operations for efficiency
    sin_t = np.sin(t)
    cos_t = np.cos(t)
    sin_theta = np.sin(theta)
    cos_theta = np.cos(theta)
    bsq = b**2
    dsq = d**2

    # shape = (3, n_points_time, 1) -> [xyz][time][constant with respect to the angle theta along the aperture]
    gamma = np.exp(b*t)*np.array([
        d*sin_t, d*cos_t, 
        np.full(n_points_time, z).reshape(n_points_time, 1)
    ])

    # Defining the normal and binormal vectors along the frennet frame for all time points and angles about the aperture
    # 3 x n_points_time x 1
    N: np.array = np.array([
        b*cos_t - sin_t,
        -b*sin_t - cos_t,
        np.zeros(n_points_time).reshape(n_points_time, 1)
    ])/np.sqrt(b**2 + 1)

    B: np.array = np.array([
        b*z*(b*sin_t + cos_t),
        b*z*(b*cos_t - sin_t),
        d*(bsq + 1)
    ]).reshape(n_points_time, 1)/np.sqrt(
        (bsq + 1)*((bsq + 1)*dsq + bsq*(z**2))
    )

    # Define the rotation matrix for the aperture given that psi 
    # is the rotation angle about the B axis in the local frennet frame
    R: np.array = np.array([
        [np.cos(psi), -np.sin(psi), 0],
        [np.sin(psi), np.cos(psi), 0],
        [0, 0, 1]
    ])
    
    # Define the unrotated and unscaled generating curve
    # Shape = (1, 1, n_points_aperture) | constant for [xyz][time], [changes for the angle]
    GC: np.array = (a*sin_theta*np.cos(phi) + cos_theta*np.sin(phi)).reshape(1, 1, n_points_aperture)*N 

    return N

In [153]:
snail = make_contreras_snail()
print(snail.shape)

C:\Users\caleb\AppData\Local\Temp\ipykernel_11892\2796418026.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  B: np.array = np.array([


ValueError: cannot reshape array of size 3 into shape (10000,1)